In [6]:
def unembed(n: int, min_v: float, max_v: float, dim: int) -> float:
    step_size = float((max_v - min_v) / (dim-1))
    v = min_v + n * step_size
    return v

In [7]:
for i in range(0,8):
    print(unembed(i,-4,4,8))

-4.0
-2.857142857142857
-1.7142857142857144
-0.5714285714285716
0.5714285714285712
1.7142857142857135
2.8571428571428568
4.0


In [11]:
def embed(v, min_v, max_v, dim):
    """Возвращает бинарный вектор, длины dim"""
    step_size = dim / (max_v - min_v)
    n = int(max(0, min(dim, (v - min_v) * step_size)))
    # result = np.zeros(dim, dtype="float32")
    result = np.full(dim, 0, dtype="float32")
    result[n] = 0.9
    return result

In [12]:
import numpy as np
for i in range(-4,4):
    print(embed(i,-4,4,8))

[0.9 0.  0.  0.  0.  0.  0.  0. ]
[0.  0.9 0.  0.  0.  0.  0.  0. ]
[0.  0.  0.9 0.  0.  0.  0.  0. ]
[0.  0.  0.  0.9 0.  0.  0.  0. ]
[0.  0.  0.  0.  0.9 0.  0.  0. ]
[0.  0.  0.  0.  0.  0.9 0.  0. ]
[0.  0.  0.  0.  0.  0.  0.9 0. ]
[0.  0.  0.  0.  0.  0.  0.  0.9]
